In [1]:
import pandas as pd
import datetime as dt
import math
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler

In [4]:
df = pd.read_csv("C:\\Users\\Murat\\Desktop\\ÖDEVLER_MİUUL\\data\\amazon_review.csv")

In [5]:
def data_sum(df):
    print("################################ DATASET OVERVİEW ####################################")
    print(30*"#"+" Number of Variables ".center(20)+30*"#")
    print(df.shape[1])
    print(30*"#"+" Number of Observations ".center(20)+30*"#")
    print(df.shape[0])
    print(30*"#"+" Type of Variables ".center(20)+30*"#")
    print(df.dtypes)
    print(30*"#"+" Top 5 Observations ".center(20)+30*"#")
    print(df.head())
    print(30*"#"+" Last 5 Observations ".center(20)+30*"#")
    print(df.tail())     
    print(30*"#"+" Number of Null Observations ".center(20)+30*"#")
    print(df.isnull().sum())
    print(30*"#"+" Summary Statistic of Data ".center(20)+30*"#")
    print(df.describe([0, 0.05,0.25, 0.50, 0.75,0.95, 0.99, 1]).T)  

In [6]:
data_sum(df)

################################ DATASET OVERVİEW ####################################
############################## Number of Variables ##############################
12
############################## Number of Observations ##############################
4915
############################## Type of Variables  ##############################
reviewerID         object
asin               object
reviewerName       object
helpful            object
reviewText         object
overall           float64
summary            object
unixReviewTime      int64
reviewTime         object
day_diff            int64
helpful_yes         int64
total_vote          int64
dtype: object
############################## Top 5 Observations ##############################
       reviewerID        asin  reviewerName helpful  \
0  A3SBTW3WS4IQSN  B007WTAJTO           NaN  [0, 0]   
1  A18K1ODH1I2MVB  B007WTAJTO          0mie  [0, 0]   
2  A2FII3I2MBMUIA  B007WTAJTO           1K3  [0, 0]   
3   A3H99DFEG68SR  B007WTAJTO 

In [7]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.0,best deal around,1373673600,2013-07-13,513,0,0


# Tarihe Göre Ağırlıklı Puan Hesaplama

###  Adım 1: Ürünün ortalama puanını hesaplayınız.

In [8]:
df["overall"].mean()

4.587589013224822

## Adım 2: Tarihe göre ağırlıklı puan ortalamasını hesaplayınız.


#### 2.1 -reviewTime değişkenini tarih değişkeni olarak değişken tipini değiştirmekteyiz.

In [9]:
df["reviewTime"] = df["reviewTime"].apply(pd.to_datetime)

#### 2.2 reviewTime'ın max değerini current_date olarak belirlemekteyiz.

In [10]:
current_date =df["reviewTime"].max()

### 2.3 Her bir puan-yorum tarihi ile current_date'in farkını gün cinsinden ifade ederek yeni değişken oluşturmanız ve gün cinsinden ifade edilendeğişkeni quantile fonksiyonu ile 4'e bölüp (3 çeyrek verilirse 4 parça çıkar) çeyrekliklerden gelen değerlere göre ağırlıklandırma yapmanız gerekir. Örneğin q1 = 12 ise ağırlıklandırırken 12 günden az süre önce yapılan yorumların ortalamasını alıp bunlara yüksek ağırlık vermek gibi.

In [11]:
df["days"] = (current_date - df["reviewTime"]).dt.days

In [12]:
df["days"].quantile([0.75,0.50,0.25])

0.75    600.0
0.50    430.0
0.25    280.0
Name: days, dtype: float64

In [13]:
def weighted_sorting_score(dataframe , w1 = 28 , w2 = 26 , w3 = 24 , w4 = 22):
    return dataframe.loc[(dataframe['days'] <= 280), 'overall'].mean() * w1 / 100 + \
        dataframe.loc[(dataframe['days'] > 280) & (dataframe['days'] <= 430) , 'overall'].mean() * w2 /100 + \
        dataframe.loc[(dataframe['days'] > 430) & (dataframe['days'] <= 600) , 'overall'].mean() * w3 / 100 + \
        dataframe.loc[(dataframe['days'] > 600) , 'overall'].mean() * w4 / 100

In [14]:
weighted_sorting_score(df,w1 = 28 , w2 = 26 , w3 = 24 , w4 = 22)

4.595593165128118

## 3: Ağırlıklandırılmış puanlamada her bir zaman diliminin ortalamasını karşılaştırıp yorumlayınız.

In [15]:
print(df.loc[df["days"] <= 280, "overall"].mean())

print(df.loc[(df["days"] > 280) & (df["days"] <= 430), "overall"].mean())

print(df.loc[(df["days"] > 430) & (df["days"] <= 600), "overall"].mean())

print(df.loc[(df["days"] > 600), "overall"].mean())

4.6957928802588995
4.636140637775961
4.571661237785016
4.4462540716612375


Oy ortalamaları ilerleyen zaman içerisinde artış göstermiştir.

# GÖREV2 Adım 2: score_pos_neg_diff, score_average_rating ve wilson_lower_bound skorlarını hesaplayıp veriye ekleyiniz.

In [16]:
df["helpful_no"] =df["total_vote"]-df["helpful_yes"]

- score_pos_neg_diff, score_average_rating ve wilson_lower_bound skorlarını hesaplayabilmek için score_pos_neg_diff,
score_average_rating ve wilson_lower_bound fonksiyonlarını tanımlayınız

In [17]:
def score_pos_neg_diff(up,down):
    return up - down

In [18]:
def score_average_rating(up,down):
    if up + down == 0:
        return 0
    return up/ (up + down)

In [19]:
def wilson_lower_bound(up, down, confidence=0.95):
    """
    Wilson Lower Bound Score hesapla

    - Bernoulli parametresi p için hesaplanacak güven aralığının alt sınırı WLB skoru olarak kabul edilir.
    - Hesaplanacak skor ürün sıralaması için kullanılır.
    - Not:
    Eğer skorlar 1-5 arasıdaysa 1-3 negatif, 4-5 pozitif olarak işaretlenir ve bernoulli'ye uygun hale getirilebilir.
    Bu beraberinde bazı problemleri de getirir. Bu sebeple bayesian average rating yapmak gerekir.

    Parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence

    Returns
    -------
    wilson score: float

    """
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)


## 2.2: score_pos_neg_diff'a göre skorlar oluşturunuz. Ardından; df içerisinde score_pos_neg_diff ismiyle kaydediniz

In [20]:
df["helpful_no"] =df["total_vote"]-df["helpful_yes"]

In [21]:
df["score_pos_neg_diff"] = df.apply(lambda x: score_pos_neg_diff(x["helpful_yes"], x["helpful_no"]), axis=1)
df.head()
    

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days,helpful_no,score_pos_neg_diff
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0,137,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,408,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0,714,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,381,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.0,best deal around,1373673600,2013-07-13,513,0,0,512,0,0


### 2.3: score_average_rating'a göre skorlar oluşturunuz. Ardından; df içerisinde score_average_rating ismiyle kaydediniz.

In [22]:
df["score_average_rating"] = df.apply(lambda x: score_average_rating(x["helpful_yes"], x["helpful_no"]), axis=1)
df.head()
   

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days,helpful_no,score_pos_neg_diff,score_average_rating
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0,137,0,0,0.0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,408,0,0,0.0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0,714,0,0,0.0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,381,0,0,0.0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.0,best deal around,1373673600,2013-07-13,513,0,0,512,0,0,0.0


In [23]:
df["score_average_rating"] = df.apply(lambda x: wilson_lower_bound(x["helpful_yes"], x["helpful_no"]), axis=1)
df.head()
   

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days,helpful_no,score_pos_neg_diff,score_average_rating
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0,137,0,0,0.0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,408,0,0,0.0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0,714,0,0,0.0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,381,0,0,0.0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.0,best deal around,1373673600,2013-07-13,513,0,0,512,0,0,0.0


### Adım 3: 20 Yorumu belirleyiniz ve sonuçları Yorumlayınız

• wilson_lower_bound'a göre ilk 20 yorumu belirleyip sıralayanız.
• Sonuçları yorumlayınız.

In [24]:
df.sort_values("wilson_lower_bound", ascending=False).head(20)
#Wilson yöntemi ile daha dikkate değer sıralamalar olduğunu görmekteyiz

KeyError: 'wilson_lower_bound'